In [8]:
import csv
from collections import defaultdict

# 1.1 - Read Fire Pokémon Above Level 40
def read_fire_pokemon(input_file, output_file):
    """Calculates and writes the percentage of Fire-type Pokémon at level 40+."""
    num_fire = 0
    num_fire_level_40_plus = 0

    with open(input_file, 'r', newline='') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            if row['type'].strip().lower() == "fire":
                num_fire += 1
                if int(float(row['level'])) >= 40:
                    num_fire_level_40_plus += 1

    percentage = round((num_fire_level_40_plus / num_fire) * 100) if num_fire > 0 else 0

    with open(output_file, 'w') as file:
        file.write(f"Percentage of fire type Pokemons at or above level 40 = {percentage}\n")


# 1.2 - Fill Missing "type" Column Values
def fill_missing_types(input_file, output_file):
    """Fills missing Pokémon 'type' based on the most common 'type' for each 'weakness'."""
    pokemon_data = []
    weakness_to_type = defaultdict(list)

    # Read Pokémon data
    with open(input_file, 'r', newline='') as file:
        csv_reader = csv.DictReader(file)
        pokemon_data = list(csv_reader)

    # Build weakness-to-type mapping
    for pokemon in pokemon_data:
        weakness = pokemon["weakness"].strip().lower()
        poke_type = pokemon["type"].strip().lower()
        if weakness and poke_type:
            weakness_to_type[weakness].append(poke_type)

    # Determine most common type for each weakness
    most_common_type = {
        weakness: max(set(types), key=types.count) for weakness, types in weakness_to_type.items()
    }

    # Fill missing type values
    for pokemon in pokemon_data:
        if not pokemon["type"].strip():  # Only fill if empty
            weakness = pokemon["weakness"].strip().lower()
            if weakness in most_common_type:
                pokemon["type"] = most_common_type[weakness]
                print(f"Filled type for {pokemon['name']}: {pokemon['type']}")  # Debug

    # Write updated dataset to pokemonResult.csv
    with open(output_file, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=pokemon_data[0].keys())
        writer.writeheader()
        writer.writerows(pokemon_data)


# 1.3 - Fill Missing Attack, Defense, and HP Stats
def fill_missing_stats(output_file):
    """Fills missing 'atk', 'def', and 'hp' values using average stats based on level."""
    pokemon_data = []
    level_threshold = 40
    high_stats = {'atk': [], 'def': [], 'hp': []}
    low_stats = {'atk': [], 'def': [], 'hp': []}

    # Read data from pokemonResult.csv
    with open(output_file, 'r', newline='') as file:
        csv_reader = csv.DictReader(file)
        pokemon_data = list(csv_reader)

    # Categorize Pokémon stats by level
    for pokemon in pokemon_data:
        try:
            level = int(float(pokemon["level"].strip()))
            for stat in ["atk", "def", "hp"]:
                if pokemon[stat].strip():  # Ignore empty values
                    value = float(pokemon[stat].strip())
                    if level > level_threshold:
                        high_stats[stat].append(value)
                    else:
                        low_stats[stat].append(value)
        except ValueError:
            continue

    # Compute average stats
    avg_high = {stat: round(sum(values) / len(values), 1) if values else 0 for stat, values in high_stats.items()}
    avg_low = {stat: round(sum(values) / len(values), 1) if values else 0 for stat, values in low_stats.items()}

    # Fill missing values
    for pokemon in pokemon_data:
        try:
            level = int(float(pokemon["level"].strip()))
            avg_stats = avg_high if level > level_threshold else avg_low
            for stat in ["atk", "def", "hp"]:
                if not pokemon[stat].strip():  # Fill only if missing
                    pokemon[stat] = avg_stats[stat]
                    print(f"Filled {stat} for {pokemon['name']} with {avg_stats[stat]}")  # Debug
        except ValueError:
            continue

    # Overwrite pokemonResult.csv with updated data
    with open(output_file, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=pokemon_data[0].keys())
        writer.writeheader()
        writer.writerows(pokemon_data)


# 1.4 - Map Pokémon Type to Personality Traits
def generate_type_to_personality_mapping(input_file, output_file):
    """Generates a mapping of Pokémon type to personality traits."""
    type_to_personality = defaultdict(set)

    with open(input_file, 'r', newline='') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            if row["type"].strip() and row["personality"].strip():
                type_to_personality[row["type"].strip().lower()].add(row["personality"].strip().lower())

    # Write type-to-personality mapping
    with open(output_file, "w") as file:
        file.write("Pokemon type to personality mapping:\n\n")
        for poke_type, personalities in sorted(type_to_personality.items()):
            file.write(f"  {poke_type}: {', '.join(sorted(personalities))}\n")


# 1.5 - Compute Average HP for Stage 3 Pokémon
def compute_average_hp_stage_3(input_file, output_file):
    """Computes the average HP of Pokémon at stage 3.0."""
    hp_values = []

    with open(input_file, 'r', newline='') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            if row["stage"] == "3.0" and row["hp"].strip():
                hp_values.append(float(row["hp"]))

    avg_hp = round(sum(hp_values) / len(hp_values)) if hp_values else 0

    # Write average HP to file
    with open(output_file, "w") as file:
        file.write(f"Average hit point for Pokemons of stage 3.0 = {avg_hp}\n")


# Main function to execute all parts
def main():
    """Executes all Pokémon processing tasks."""
    input_file = "pokemonTrain.csv"
    result_file = "pokemonResult.csv"  # Single output file

    # Run all the tasks in order
    read_fire_pokemon(input_file, "pokemon1.txt")
    fill_missing_types(input_file, result_file)  # Writes to pokemonResult.csv
    fill_missing_stats(result_file)  # Modifies the same file
    generate_type_to_personality_mapping(result_file, "pokemon4.txt")
    compute_average_hp_stage_3(result_file, "pokemon5.txt")

# Run the program
